<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Deploy and score an AutoAI model using the REST API</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook will take you through the steps of using the REST API to deploy and score an AutoAI model made with the <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/50fa9246181026cd7ae2a5bc7ea444e6" target="_blank" rel="noopener noreferrer">Bank Marketing</a> data set.

**<a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-overview.html" target="_blank" rel="noopener noreferrer">AutoAI</a>** is a tool on Watson Studio that automatically analyzes data and builds model pipelines with an uploaded data set. AutoAI employs algorithms to infer patterns from your data, prepare the data, and modify the data, all the while requiring minimal user input. The AutoAI process follows the following steps
- Data preprocessing
- Automated model selection
- Automated feature engineering
- Hyperparameter optimization

After the tool has finished building the model pipelines, it will list all the models on a leaderboard, ranked by a metric of the user's choosing such as accuracy, precision, or ROC-AUC (based on the prediction type).

You will use the Bank Marketing data set available on the <a href="https://archive.ics.uci.edu/ml/datasets/Bank+Marketing" target="_blank" rel="noopener noreferrer">UCI Machine Learning Repository</a>. This data deals with a Portuguese banking institution's marketing campaign, which used phone calls to determine whether customers would subscribe to a term deposit.

The notebook uses Python and the Watson Machine Learning REST API to manage the deployments.

## 1. Setting up <a id="setup"></a>

**Action:** Before you proceed with the steps in this notebook, please ensure that you've completed the following:
1. Get a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener noreferrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create an instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">here</a>).
2. Go to the <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/50fa9246181026cd7ae2a5bc7ea444e6" target="_blank" rel="noopener noreferrer">Bank Marketing</a> data set in the Watson Studio Gallery.
3. Load the .csv file as a Data Asset in your project by clicking the **Add to Project** button in the upper-right corner of the page. Select your project name and click **Add**. You can now view the data set in your project.
4. In the next step, you will <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-build.html" target="_blank" rel="noopener noreferrer">build an AutoAI model</a>.

## 2. Build an AutoAI model

1. In your Watson Studio project, click **Add to project**. Then, click **AutoAI Experiment**.
2. Make sure you have **From Sample** selected. Specify a name and description for your experiment and click **Create**.
3. To add the training data, choose the Bank Marketing .csv file from your project. You'll now see an option to select the column to predict. 
4. Select the column named `y`. Based on analyzing a subset of the data set, AutoAI will choose a default model type: binary classification. 
5. Click **Run Experiment** to begin the model pipeline creation.
6. When the pipeline generation process completes, you can view the leading model candidates and <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-view-results.html" target="_blank" rel="noopener noreferrer">evaluate</a> them before saving a pipeline as a model. The default ranking metric for binary classification models is the area under the ROC curve, so with Ranking based on `ROC AUC`, select the top pipeline. Click **Save as model**. 

## 3. Deploy the model

1. Once you've saved the model from your AutoAI Experiment, go back to the **Assets** tab of your project. You'll find the model you saved under the **Models** section. 
2. Click on your model, navigate to the **Deployments** tab, and select **Add Deployment**. 
3. Select your deployment once it's ready and copy the `Scoring End-point` under the **Implementation** tab.

In [1]:
scoring_url = "Copy the scoring end-point here"

## 4. Score

### Load the data

1. Load the Bank Marketing .csv file into your notebook. Click the Data icon on the notebook action bar. Scroll through your list of assets to find the Bank Marketing .csv that you added to your project [earlier](#setup). **Note**: If you can't find the data asset, reload your notebook. To load the data into a DataFrame, click in the next code cell and select **Insert to code > Insert Pandas DataFrame** under the file name.
2. Rename `df_data_x` to `df_data_1`.
3. Run the cell.

In [2]:
# Load Bank Marketing data as dataframe
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_985f24d93a7243aeb65f8aba59f75706 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0xUoGllZR1Mibim6B4wEd26AllWNzgv64u1MjDkTBwrE',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_985f24d93a7243aeb65f8aba59f75706.get_object(Bucket='wastonmachinelearningcommunitynot-donotdelete-pr-pnbylejs5jpqwu',Key='bank.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


Here, you can see a preview of the Bank Marketing data set.

In the next step, you will learn how to use the REST API.

### REST API

First, you'll have to <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html#python-example" target="_blank" rel="noopener noreferrer">generate an IAM token</a> in order to use the Watson Machine Learning REST API. The IAM token requires an `apikey` for authentication, so you may need to generate new Service Credentials if you do not already have one.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-get-wml-credentials.html" target="_blank" rel="noopener noreferrer">Service Credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information.

Once you have your credentials, copy the `apikey` and the `instance_id` and paste them in the cell below.

In [3]:
# Paste your Watson Machine Learning service apikey and isntance_id here
apikey = "****"
ml_instance_id = "****"

Run the following cell to generate the IAM token.

In [4]:
import requests

# Get an IAM token from IBM Cloud
url     = "https://iam.bluemix.net/oidc/token"
headers = { "Content-Type" : "application/x-www-form-urlencoded" }
data    = "apikey=" + apikey + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"
IBM_cloud_IAM_uid = "bx"
IBM_cloud_IAM_pwd = "bx"
response  = requests.post( url, headers=headers, data=data, auth=( IBM_cloud_IAM_uid, IBM_cloud_IAM_pwd ) )
iam_token = response.json()["access_token"]

Prepare the scoring payload with the records to score. The HTTP header will require the `instance_id` from your service credentials.

In [5]:
import urllib3, json
from pprint import pprint

# NOTE: generate iam_token and retrieve ml_instance_id based on provided documentation
header = {"Content-Type": "application/json", "Authorization": "Bearer " + iam_token, "ML-Instance-ID": ml_instance_id}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {
    "input_data": [{
        "fields": ["age", "job", "marital", "education", "default", "balance", "housing",
       "loan", "contact", "day", "month", "duration", "campaign", "pdays",
       "previous", "poutcome", "y"],
        "values": [[30, "unemployed", "married", "primary", "no", 1787, "no", "no", "cellular", 19, "oct", 79, 1, -1, 0, "unknown", "no"],
                   [20, "student", "single", "secondary", "no", 502, "no", "no", "cellular", 30, "apr", 261, 1, -1, 0, "unknown", "yes"]]
    }]
}
pprint(payload_scoring)

{'input_data': [{'fields': ['age',
                            'job',
                            'marital',
                            'education',
                            'default',
                            'balance',
                            'housing',
                            'loan',
                            'contact',
                            'day',
                            'month',
                            'duration',
                            'campaign',
                            'pdays',
                            'previous',
                            'poutcome',
                            'y'],
                 'values': [[30,
                             'unemployed',
                             'married',
                             'primary',
                             'no',
                             1787,
                             'no',
                             'no',
                             'cellular',
                  

Now, you can use the REST API, the scoring endpoint, and the payload you've prepared to score the deployed model.

In [6]:
# NOTE: obtain scoring url of deployment

response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)
print("Scoring response")
pprint(json.loads(response_scoring.text))

Scoring response
{'predictions': [{'fields': ['prediction', 'probability'],
                  'values': [[0.0, [0.9732180907567131, 0.02678190924328699]],
                             [1.0,
                              [0.24594307548171945, 0.7540569245182805]]]}]}


You can see that the first client is predicted to have **not been subscribed** a term deposit with a probability of 0.97. The second client is predicted to have been **subscribed** a term deposit with a probability of 0.75.

## 5. Summary and next steps

You have successfully completed this notebook! 

You learned how to use deploy and score an AutoAI model using the REST API. 

Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Citation

Moro, S., Cortez, P., & Rita, P. (2014). A data-driven approach to predict the success of bank telemarketing. Decision Support Systems, 62, 22-31.

### Authors

**Ananya Kaushik** is a Data Scientist at IBM.

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>